In [39]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import datetime
from tqdm.notebook import tqdm
from IPython.core.debugger import set_trace

In [34]:
dates_list = ['15/02/2022','25/01/2022','14/01/2022','18/12/2021','14/12/2021','26/11/2021','12/11/2021','02/11/2021',
              '14/09/2021','13/08/2021','09/07/2021','18/06/2021','28/05/2021','11/05/2021','20/04/2021','13/04/2021',
              '23/03/2021','09/03/2021','23/02/2021','02/02/2021','20/01/2021','12/01/2021','14/12/2020','17/11/2020',
              '03/11/2020','13/10/2020','28/09/2020','25/09/2020','01/09/2020','18/08/2020','06/08/2020','24/06/2020',
              '19/05/2020','06/05/2020','21/04/2020','31/03/2020','23/03/2020','15/03/2020','12/03/2020']
extra_filts = ' since:{0} until:{1} lang:nl'
key_words = list(map(lambda x: x.strip(), '(corona maatregel) OR (corona maatregelen) OR coronamaatregelen OR coronamaatregel OR coronaregel OR coronaregels OR coronamaatregels OR (corona maatregels)\
    OR (covid maatregel) OR (covid maatregelen) OR covidmaatregelen OR covidmaatregel OR covidregel OR covidregels OR covidmaatregels OR (covid maatregels)\
    OR (covid19 maatregel) OR (covid19 maatregelen) OR covid19maatregelen OR covid19maatregel OR covid19regel OR covid19regels OR covid19maatregels OR (covid19 maatregels)\
    OR (covid-19 maatregel) OR (covid-19 maatregelen) OR covid-19maatregelen OR covid-19maatregel OR covid-19regel OR covid-19regels OR covid-19maatregels OR (covid-19 maatregels)\
    OR (sars-cov-2 maatregel) OR (sars-cov-2 maatregelen) OR sars-cov-2maatregelen OR sars-cov-2maatregel OR sars-cov-2regel OR sars-cov-2regels OR sars-cov-2maatregels OR (sars-cov-2 maatregels)\
    OR lockdown OR persconferentie OR (1,5 meter) OR anderhalvemeter OR mondkapje OR mondkap OR mondmasker OR masker OR blijfthuis OR thuiswerken OR quarantaine OR thuisquarantaine OR (thuis quarantaine) OR testenvoortoegang OR (testen voor toegang)'.split('OR')))


In [47]:
def append_to_query_str(q_str, k_word):
    return '%s OR %s'%(q_str, k_word) if len(q_str) != 0 else k_word

def constr_query_groups(key_words, extra_filts):
    '''Divides keywords over several queries so the max query length
    and OR count set by Twitter are never reached'''
    query_lim = 400
    or_lim = 49
    q_groups = []
    q_str = ''
    
    i=0
    for k_word in key_words:
                  
        if not ((len(append_to_query_str(q_str, k_word)) + len(extra_filts)) > query_lim) and not i-1 > 50:
            q_str = append_to_query_str(q_str, k_word)
        else:
            q_groups.append(q_str +extra_filts)
            q_str = k_word
            i=0
    q_groups.append(q_str+extra_filts)
    
    return q_groups
        

In [42]:
# Creating list to append tweet data to
tweets_list = []
query_groups = constr_query_groups(key_words, extra_filts)


for date in tqdm(dates_list):

    date_1 = datetime.datetime.strptime(date, "%d/%m/%Y")
    end_date = date_1 + datetime.timedelta(days=7)
    end_date = datetime.datetime.strftime(end_date, "%Y-%m-%d")
    
    for q in tqdm(query_groups):
    
        for i, tweet in enumerate(sntwitter.TwitterSearchScraper(q.format(
            date, end_date)).get_items()):
            tweets_list.append(tweet)

        
# Creating a dataframe from the tweets list above
tweets_df = pd.DataFrame(tweets_list)

  0%|          | 0/39 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [45]:
tweets_df.describe()

,id,replyCount,retweetCount,likeCount,quoteCount,conversationId,inReplyToTweetId
count,3.345000e+03,3345.000000,3345.000000,3345.000000,3345.000000,3.345000e+03,5.370000e+02
mean,1.371923e+18,0.874738,4.832586,11.188042,0.511809,1.371125e+18,1.373904e+18
std,8.436612e+16,4.536222,26.189799,67.656320,3.049815,8.443525e+16,8.835174e+16
min,1.032262e+18,0.000000,0.000000,0.000000,0.000000,1.032262e+18,1.107356e+18
25%,1.312002e+18,0.000000,0.000000,0.000000,0.000000,1.311969e+18,1.302894e+18
50%,1.357010e+18,0.000000,0.000000,0.000000,0.000000,1.356923e+18,1.362001e+18
75%,1.461067e+18,1.000000,1.000000,2.000000,0.000000,1.459513e+18,1.463236e+18
max,1.495844e+18,103.000000,538.000000,1561.000000,64.000000,1.495844e+18,1.495364e+18


In [46]:
from pathlib import Path  
filepath = Path('data/maatregelen_tweets_per_persco_7dagen.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
tweets_df.to_csv(filepath)

mogelijk mee te nemen hashtags: hetisklaar, corona, covid19, covid, covid19nl, coronavirus